In [1]:
!pip install transformers datasets scikit-learn --quiet

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BartTokenizer, BartForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time
import os


In [4]:
# Load dataset
df = pd.read_csv('/content/Reviews.csv', on_bad_lines='warn', engine='python')
df = df[['Text', 'Score']].dropna()
df = df[df['Score'] != 3]  # Remove neutral reviews
df['label'] = df['Score'].apply(lambda x: 1 if x > 3 else 0)  # 1=positive, 0=negative


/tmp/ipython-input-4-2071192347.py:2: ParserWarning: Skipping line 211038: unexpected end of data

  df = pd.read_csv('/content/Reviews.csv', on_bad_lines='warn', engine='python')
/tmp/ipython-input-4-2071192347.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['Score'].apply(lambda x: 1 if x > 3 else 0)  # 1=positive, 0=negative


In [5]:
df = df.sample(15000)

In [7]:
# This cell is intentionally left empty as its contents have been moved to the next cell.

In [11]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Text'], df['label'], test_size=0.2, random_state=42
)
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)


In [12]:
class AmazonDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = AmazonDataset(train_encodings, train_labels.tolist())
val_dataset = AmazonDataset(val_encodings, val_labels.tolist())

In [13]:
model = BartForSequenceClassification.from_pretrained(
    'facebook/bart-base',
    num_labels=2
)

# Training arguments
os.environ["WANDB_DISABLED"] = "true"  # Disable W&B tracking

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/BART_Sentiment/results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    eval_strategy="epoch",
    logging_dir='/content/drive/MyDrive/BART_Sentiment/logs',
    logging_steps=10,
    save_strategy="no"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [15]:
model = BartForSequenceClassification.from_pretrained(
    'facebook/bart-base',
    num_labels=2
)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [17]:
# Train model
start_time = time.time()
trainer.train()
training_time = time.time() - start_time
print(f"Training time: {training_time:.2f} seconds")

Epoch,Training Loss,Validation Loss
1,0.152300,0.170447
2,0.093000,0.170226


Training time: 679.56 seconds


AttributeError: 'tuple' object has no attribute 'argmax'

In [18]:
# Evaluation
start_test = time.time()
predictions = trainer.predict(val_dataset)
test_time = time.time() - start_test

y_pred = predictions.predictions[0].argmax(axis=-1)
y_true = val_labels[:len(y_pred)]

# Calculate metrics
acc = accuracy_score(y_true, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')

print(f"Accuracy: {acc}")
print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"F1 Score: {f1}")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_time:.2f} seconds")

Accuracy: 0.9573333333333334
Precision: 0.9729093050647821
Recall: 0.9767441860465116
F1 Score: 0.974822974036192
Training Time: 679.56 seconds
Testing Time: 24.75 seconds
